<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
df = pd.read_csv('/datasets/insurance.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [5]:
df.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

Изменим тип данных в столбцах Пол, Возраст, Члены семьи и Страховые выплаты на целочисленный ввиду отсуствия дробных частей. 

In [6]:
df['Пол'] = pd.to_numeric(df['Пол'], downcast='integer')
df['Возраст'] = pd.to_numeric(df['Возраст'], downcast='integer')
df['Члены семьи'] = pd.to_numeric(df['Члены семьи'], downcast='integer')
df['Страховые выплаты'] = pd.to_numeric(df['Страховые выплаты'], downcast='integer')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int8   
 1   Возраст            5000 non-null   int8   
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int8   
 4   Страховые выплаты  5000 non-null   int8   
dtypes: float64(1), int8(4)
memory usage: 58.7 KB


Данные загружены и не имеют аномалий. 

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

## Алгоритм преобразования

**Алгоритм**

    
1. Строю линейную регрессию на начальных данных и нахожу метрику R2.


2. Создаю случайную матрицу.


3. Проверяю её на обратимость.


4. Умножаю начальные данные на случайную матрицу.


5. Строю линейную регрессию на основе данных, умноженных на случайную матрицу, и нахожу метрику R2.


6. Сравниваю результаты между собой метрики R2.

## Проверка алгоритма

1. Строю линейную регрессию на начальных данных и нахожу метрику R2.

In [7]:
features = df.drop('Страховые выплаты',axis=1)
target = df['Страховые выплаты']

In [8]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

In [9]:
class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0
    
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
R2 = r2_score(target, predictions)
print(R2)

0.42494550286668


2. Создаю случайную матрицу.

3. Проверяю её на обратимость.

4. Умножаю начальные данные на случайную матрицу.

5. Строю линейную регрессию на основе данных, умноженных на случайную матрицу, и нахожу метрику R2.


In [10]:
class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]
        
        X_d = X.shape[1]
        Z = np.random.normal(size=(X_d, X_d))
        
        self.Z = Z
        self.X = X
        return self
    
    def check(self):
        Z_rev = np.linalg.inv(Z)
        if np.allclose(np.dot(self.Z, Z_rev), np.eye(self.Z.shape[0])) is True:
            ans = "Матрица обратима."
        else:
            ans = "Ошибка: матрица не обратима."
        return ans
    
    def transform(self, X, y=None):  
        return X.dot(self.Z)

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0
            
model = LinearRegression()
model.fit(features, target)
predictions_new = model.predict(features)
R2_new = r2_score(target, predictions_new)
#print(ans)
print(R2_new)

0.42494550286668


6. Сравниваю результаты между собой метрики R2.

In [12]:
answers = pd.DataFrame({'Линейная регрессия': [R2],
                      'Линейная регрессия преобразованая': [R2_new]})
answers

,Линейная регрессия,Линейная регрессия преобразованая
0,0.424946,0.424946
